In [1]:
import pandas as pd
import csv

In [2]:
df = pd.read_excel(io=r"C:\Users\Chris\OneDrive\Documents\Excel\New_Money.xlsx", sheet_name=0, engine='openpyxl')

In [3]:
df['Date'] = df['Date'].astype(str)
df['Date'] = df['Date'].str.replace('-', '')

In [4]:
def convert_lines(data: pd.DataFrame) -> pd.DataFrame:
    formatted_rows = []
    id_num = 1

    for row in data.iterrows():

        date, expense, amount, cat, subcat, recip, project, note = row[1].values
        time = ''
        currency = 'USD'
        periodic = ''
        income = ''
        add = False

        if subcat == 'Cash Deposit' and amount < 0:
            income = 'Checking'
            add = True
        elif subcat == 'Cash Withdrawal' and amount < 0:
            income = 'Cash'
            add = True
        elif subcat == 'Payment' and amount < 0:
            income = recip
            recip = ''
            add = True
        elif cat == 'Initial':
            cat = 'SYSTEM'
            subcat = 'INIT_AMOUNT'
            date = 10100101
            income = expense
            expense = ''
            recip = ''
            add = True
        elif cat == 'Income':
            income = expense
            expense = ''
            add = True
        elif cat != 'Transfer' and amount < 0:
            add = True

        if add:
            formatted_rows.append(
                {
                    'Id': id_num,
                    'Currency': currency,
                    'Amount': abs(amount),
                    'Category': cat,
                    'Sub-Category': subcat,
                    'Date': date,
                    'Expense(Transfer Out)': expense,
                    'Income(Transfer In)': income,
                    'Note': note,
                    'Periodic': periodic,
                    'Project': project,
                    'Payee/Payer': recip,
                    'uid': id_num,
                    'Time': time
                }
            )
            id_num += 1
        
    return pd.DataFrame.from_records(formatted_rows)
        

In [5]:
primary = convert_lines(df)

In [6]:
# primary.to_excel('to_andro.xlsx', index=False)
# primary.to_csv('AndroMoney.csv', index=False)
primary.to_csv('AndroMoney.csv', sep=",", quotechar='"', index=False, quoting=csv.QUOTE_ALL)